In [ ]:
import keras
import numpy as np
from keras import callbacks,optimizers,losses
from sklearn.metrics import roc_curve,roc_auc_score
import matplotlib.pyplot as plt
import pandas as pd

#.py files
import generate_data as generator
import models as model_builder

# Classification

In [ ]:
optimizer= optimizers.Adam(learning_rate=5e-5)
loss_function= losses.BinaryCrossentropy()
metrics=['binary_accuracy']
epochs= 50
steps_per_epoch = 10
fit_callbacks = [
    #callbacks.EarlyStopping(
     #   monitor= 'val_accuracy',
      #  min_delta= 0.1,
       # patience= 15, 
        #restore_best_weights= True),
    callbacks.CSVLogger(
        filename= 'BCDR Patching Training Results',
        separator=",", 
        append=True)
]

## Sistematic 350

In [ ]:
patch_size = 250
batch_size = 128

In [ ]:
training_data = generator.generate_patch_data('S_350_patches/train',patch_size,patch_size,batch_size,'grayscale')
validation_data = generator.generate_patch_data('S_350_patches/validation',patch_size,patch_size,batch_size,'grayscale')

In [ ]:
model = model_builder.create_patch_model(patch_size,patch_size)
model.summary()

In [ ]:
trained_model = model_builder.train_model(model,optimizer,loss_function,metrics,epochs,steps_per_epoch,training_data,validation_data,fit_callbacks)

## Random 350

In [ ]:
patch_size = 350
batch_size = 128

In [ ]:
training_data = generator.generate_patch_data('R_350_patches/train',patch_size,patch_size,batch_size,'grayscale')
validation_data = generator.generate_patch_data('R_350_patches/validation',patch_size,patch_size,batch_size,'grayscale')

In [ ]:
model = model_builder.create_patch_model(patch_size,patch_size)
model.summary()

In [ ]:
trained_model = model_builder.train_model(model,optimizer,loss_function,metrics,epochs,steps_per_epoch,training_data,validation_data,fit_callbacks)

## Sistematic 300

In [ ]:
patch_size = 300
batch_size = 256

In [ ]:
training_data = generator.generate_patch_data('S_300_patches/train',patch_size,patch_size,batch_size,'grayscale')
validation_data = generator.generate_patch_data('S_300_patches/validation',patch_size,patch_size,batch_size,'grayscale')

In [ ]:
model = model_builder.create_patch_model(patch_size,patch_size)
model.summary()

In [ ]:
trained_model = model_builder.train_model(model,optimizer,loss_function,metrics,epochs,steps_per_epoch,training_data,validation_data,fit_callbacks)

## Random 300

In [ ]:
patch_size = 300
batch_size = 256

In [ ]:
training_data = generator.generate_patch_data('R_300_patches/train',patch_size,patch_size,batch_size,'grayscale')
validation_data = generator.generate_patch_data('R_300_patches/validation',patch_size,patch_size,batch_size,'grayscale')

In [ ]:
model = model_builder.create_patch_model(patch_size,patch_size)
model.summary()

In [ ]:
trained_model = model_builder.train_model(model,optimizer,loss_function,metrics,epochs,steps_per_epoch,training_data,validation_data,fit_callbacks)

## Sistematic 250

In [ ]:
patch_size = 250
batch_size = 256

In [ ]:
training_data = generator.generate_patch_data('S_250_patches/train',patch_size,patch_size,batch_size,'grayscale')
validation_data = generator.generate_patch_data('S_250_patches/validation',patch_size,patch_size,batch_size,'grayscale')

In [ ]:
model = model_builder.create_patch_model(patch_size,patch_size)
model.summary()

In [ ]:
trained_model = model_builder.train_model(model,optimizer,loss_function,metrics,epochs,steps_per_epoch,training_data,validation_data,fit_callbacks)

## Random 250

In [ ]:
patch_size = 250
batch_size = 128

In [ ]:
training_data = generator.generate_patch_data('R_250_patches/training_data',patch_size,patch_size,batch_size,'grayscale')
validation_data = generator.generate_patch_data('R_250_patches/validation_data',patch_size,patch_size,batch_size,'grayscale')

In [ ]:
model = model_builder.create_patch_model(patch_size,patch_size)
model.summary()

In [ ]:
trained_model = model_builder.train_model(model,optimizer,loss_function,metrics,epochs,steps_per_epoch,training_data,validation_data,fit_callbacks)

## Transfer Learning

In [ ]:
patch_size = 300
batch_size = 128

In [ ]:
training_data = generator.generate_patch_data('D:/Lesion-Based/training_data',patch_size,patch_size,batch_size,'rgb')
validation_data = generator.generate_patch_data('D:/Lesion-Based/validation_data',patch_size,patch_size,batch_size,'rgb')

In [ ]:
vgg_model = keras.applications.VGG19(include_top=False,weights='imagenet',input_shape=(patch_size,patch_size,3))
vgg_model.trainable=False

In [ ]:
model = keras.models.Sequential()
model.add(vgg_model)
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(1, activation='sigmoid'))
model.summary()

In [ ]:
trained_model = model_builder.train_transferred_model(model,optimizer,loss_function,metrics,epochs,steps_per_epoch,training_data,validation_data,fit_callbacks)

In [ ]:
y_true= []
for i in range(len(validation_data)):
    for n in range(len(validation_data[i][1])):
        y_true.append(validation_data[i][1][n])
print(y_true)

In [ ]:
ypred= model.predict(validation_data)

In [ ]:
y_pred=[]
for i in range(len(ypred)):
    y_pred.append(ypred[i][0])

In [ ]:
predictions = pd.DataFrame({'ground truth':y_true,'predictions':y_pred})

In [ ]:
fpr,tpr,_= roc_curve(y_true,y_pred)

In [ ]:
print('AUC: ',roc_auc_score(y_true,y_pred))

In [ ]:
plt.plot(fpr,tpr)

In [ ]:
print(validation_data[0][1])